# AutoMM Detection - Quick Start with Foundation Model on Open Vocabulary Detection (OVD)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/autogluon/autogluon/blob/master/docs/tutorials/multimodal/object_detection/quick_start/quick_start_ovd.ipynb)
[![Open In SageMaker Studio Lab](https://studiolab.sagemaker.aws/studiolab.svg)](https://studiolab.sagemaker.aws/import/github/autogluon/autogluon/blob/master/docs/tutorials/multimodal/object_detection/quick_start/quick_start_ovd.ipynb)


In this section, our goal is to use a foundation model in object detection to detect novel classes defined by an unbounded (open) vocabulary.

## Setting up the imports
To start, let's import MultiModalPredictor:

In [1]:
!pip install autogluon.multimodal
!pip install git+https://github.com/IDEA-Research/GroundingDINO.git


  Cloning https://github.com/IDEA-Research/GroundingDINO.git to /tmp/pip-req-build-ze_55bg3
  Running command git clone --filter=blob:none --quiet https://github.com/IDEA-Research/GroundingDINO.git /tmp/pip-req-build-ze_55bg3
  Resolved https://github.com/IDEA-Research/GroundingDINO.git to commit 498048b1b2b770aca3f98430167dbbf458915c18
  Preparing metadata (setup.py) ... done


In [2]:
from autogluon.multimodal import MultiModalPredictor

## Prepare sample image
Let's use an image in Cityscape dataset to demo:

In [ ]:
from IPython.display import Image, display
from autogluon.multimodal import download

sample_image_url = "https://live.staticflickr.com/65535/49004630088_d15a9be500_6k.jpg"
sample_image_path = download(sample_image_url)

display(Image(filename=sample_image_path))

## Creating the MultiModalPredictor
We create the MultiModalPredictor and specify the problem_type to `"open_vocabulary_object_detection"`.
We set the preset as `"best_quality"`, which uses a SwinB as backbone. This preset gives us higher accuracy for detection. 
We also provide presets `"high_quality"` and `"medium_quality"` with SwinT as backbone, faster but also with lower performance.

In [ ]:
# Init predictor
predictor = MultiModalPredictor(problem_type="open_vocabulary_object_detection", presets = "best_quality")

## Inference
To run inference on the image, perform:

In [ ]:
pred = predictor.predict(
    {
        "image": [sample_image_path],
        "prompt": ["Pink notice. Green sign. One Way sign. People group. Tower crane in construction. Lamp post. Glass skyscraper."],
    },
    as_pandas=True,
)

print(pred)

The output `pred` is a `pandas` `DataFrame` that has two columns, `image` and `bboxes`.

In `image`, each row contains the image path

In `bboxes`, each row is a list of dictionaries, each one representing the prediction for an object in the image: `{"class": <predicted_class_name>, "bbox": [x1, y1, x2, y2], "score": <confidence_score>}`, for example:

In [ ]:
print(pred["bboxes"][0][0])

In [ ]:
!pip install opencv-python

To visualize results, run the following:

In [ ]:
from autogluon.multimodal.utils import Visualizer

conf_threshold = 0.2  # Specify a confidence threshold to filter out unwanted boxes
image_result = pred.iloc[0]

img_path = image_result.image  # Select an image to visualize

visualizer = Visualizer(img_path)  # Initialize the Visualizer
out = visualizer.draw_instance_predictions(image_result, conf_threshold=conf_threshold)  # Draw detections
visualized = out.get_image()  # Get the visualized image

from PIL import Image
from IPython.display import display
img = Image.fromarray(visualized, 'RGB')
display(img)

## Other Examples

You may go to [AutoMM Examples](https://github.com/autogluon/autogluon/tree/master/examples/automm) to explore other examples about AutoMM.

## Customization
To learn how to customize AutoMM, please refer to [Customize AutoMM](../../advanced_topics/customization.ipynb).

## Citation

```
@misc{liu2023grounding,
      title={Grounding DINO: Marrying DINO with Grounded Pre-Training for Open-Set Object Detection}, 
      author={Shilong Liu and Zhaoyang Zeng and Tianhe Ren and Feng Li and Hao Zhang and Jie Yang and Chunyuan Li and Jianwei Yang and Hang Su and Jun Zhu and Lei Zhang},
      year={2023},
      eprint={2303.05499},
      archivePrefix={arXiv},
      primaryClass={cs.CV}
}
```
